In [1]:
from create_kfold import create_kfold
import pandas as pd

train = pd.read_csv('../input/feedback-prize-2021/train.csv')
train_kfold, train = create_kfold(df=train, k=5)

In [5]:
from tqdm import tqdm
train_names, train_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/train'))):
    train_names.append(f.replace('.txt', ''))
    train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r', encoding='utf-8').read())
train_text_df = pd.DataFrame({'id': train_names, 'text': train_texts})
train_text_df = train_text_df.merge(train_kfold, on="id", how="left")

100%|██████████| 15594/15594 [00:02<00:00, 6526.96it/s]


In [3]:
train

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,kfold
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,1
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,1
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75,1
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,1
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...,1
...,...,...,...,...,...,...,...,...,...
144288,4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...,1
144289,4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...,1
144290,4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838,1
144291,4C471936CD75,1.618025e+12,4570.0,4922.0,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...,1


In [6]:
train_text_df

,id,text,kfold
0,4BCA0580352F,Isn't it wonderful to know that you get a chan...,2
1,29A8E6DA6F04,"One uses a car to go to the store, pick someon...",1
2,17E8BB6C550B,Driverless cars do seem like the thing today's...,1
3,319B2511943D,Have you ever thought your choice wasn't the b...,3
4,C9B1827498E4,"Imagine you're sitting at home, watching a sho...",4
...,...,...,...
15589,F2DFE12EA3B4,Many students have expressed that projects sho...,3
15590,5976F16FE9D2,In the 1960's driverless cars seemed more as l...,4
15591,D4CF0DF858F1,"Looking back on that last tough decision, do y...",1
15592,DA8115F12B50,Everyday a great many people drive their cars ...,1
